## 相机内参标定 
通过拍摄多张棋盘格照片得到相机内参。

In [2]:
import numpy as np
import cv2
import os
import matplotlib.pyplot as plt

path = 'D:\\labAAA\\printingLight\\process\\camera_set\\chessboard'
list_path = os.listdir(path)
# print(list_path)


w, h = 10, 7
square_size = 15
criteria = (cv2.TERM_CRITERIA_EPS + cv2.TERM_CRITERIA_MAX_ITER, 100, 0.001)
# 准备棋盘格的世界坐标点
objp = np.zeros((w * h, 3), np.float32)
objp[:, :2] = np.mgrid[0:w, 0:h].T.reshape(-1, 2)
objp *= square_size


grays = []
pImgs = []
objpoints = []  # 世界坐标点
imgpoints = []  # 图像像素点


# 遍历图像并检测棋盘角点
for file in list_path:
    file_name = os.path.join(path, file)
    img = cv2.imread(file_name)
    scale = 0.5  # 50%缩小
    img_resize = cv2.resize(img, (int(img.shape[1] * scale), int(img.shape[0] * scale)))
    

    gray = cv2.cvtColor(img_resize, cv2.COLOR_BGR2GRAY)
    # gray_resize = cv2.resize(gray, (int(gray.shape[1] * scale), int(gray.shape[0] * scale)))
    print('image pixel', gray.shape[::-1])
    grays.append(gray)

    ret, corners = cv2.findChessboardCorners(gray, (w, h), None)
    print(f"Chessboard found in {file}: {ret}")
    
    if ret:
        objpoints.append(objp)
        corners2 = cv2.cornerSubPix(gray, corners, (11, 11), (-1, -1), criteria)
        imgpoints.append(corners2)
        


        # 可视化检测到的角点（用 matplotlib 替代 OpenCV）
        # img_with_corners = img_resize.copy()
        # cv2.drawChessboardCorners(img_with_corners, (w, h), corners2, ret)
        #使用 matplotlib 显示
        # plt.figure(figsize=(8, 6))
        # plt.imshow(cv2.cvtColor(img_with_corners, cv2.COLOR_BGR2RGB))  # 转换为 RGB 格式
        # plt.scatter(corners2[:, 0, 0], corners2[:, 0, 1], marker='o', c='red', s=10, label="Detected Corners")  # `s`控制点大小
        # plt.title(f"Detected Corners: {file}")
        # plt.rcParams['lines.linewidth'] = 2  # 设置线条宽度
        # plt.axis("off")  # 隐藏坐标轴
        # plt.show()


# 相机标定
print("\nStarting camera calibration...")
# ret, camera_matrix, distortion_coeffs, rvecs, tvecs = cv2.calibrateCamera(
#     objpoints, imgpoints, grays[0].shape[::-1], None, None
# )
ret, camera_matrix, distortion_coeffs, rvecs, tvecs = cv2.calibrateCamera(
    objpoints, imgpoints, grays[0].shape[::-1], None, None,flags=cv2.CALIB_FIX_K3 | cv2.CALIB_FIX_K4 | cv2.CALIB_FIX_K5
)

# 打印结果
print("相机矩阵 (Camera Matrix):\n", camera_matrix)
print("\n畸变系数 (Distortion Coefficients):\n", distortion_coeffs)

# 计算重投影误差
mean_error = 0
for i in range(len(objpoints)):
    imgpoints2, _ = cv2.projectPoints(objpoints[i], rvecs[i], tvecs[i], camera_matrix, distortion_coeffs)
    error = cv2.norm(imgpoints[i].squeeze(), imgpoints2.squeeze(), cv2.NORM_L2) / len(imgpoints2)
    mean_error += error

print("\n总平均误差 (Mean Reprojection Error):", mean_error / len(objpoints))

# 保存标定结果
np.savez("calibration_result.npz", camera_matrix=camera_matrix, distortion_coeffs=distortion_coeffs, rvecs=rvecs, tvecs=tvecs)

# for file in list_path:
#     file_name = os.path.join(path, file)
#     img = cv2.imread(file_name)
#     scale = 0.5  # 50%缩小
#     img_resize = cv2.resize(img, (int(img.shape[1] * scale), int(img.shape[0] * scale)))
#     undistorted_img = cv2.undistort(img_resize, camera_matrix, distortion_coeffs)
#     plt.figure(figsize=(8, 6))
#     plt.imshow(cv2.cvtColor(undistorted_img, cv2.COLOR_BGR2RGB))  # 转换为 RGB 格式
#     plt.title(f"Detected Corners: {file}")
#     plt.axis("off")  # 隐藏坐标轴
#     plt.show()


image pixel (2736, 1824)
Chessboard found in IMG_6519.JPG: True
image pixel (2736, 1824)
Chessboard found in IMG_6520.JPG: True
image pixel (2736, 1824)
Chessboard found in IMG_6521.JPG: True
image pixel (2736, 1824)
Chessboard found in IMG_6523.JPG: True
image pixel (2736, 1824)
Chessboard found in IMG_6525.JPG: True
image pixel (2736, 1824)
Chessboard found in IMG_6526.JPG: True
image pixel (2736, 1824)
Chessboard found in IMG_6528.JPG: True
image pixel (2736, 1824)
Chessboard found in IMG_6529.JPG: True
image pixel (2736, 1824)
Chessboard found in IMG_6530.JPG: True
image pixel (2736, 1824)
Chessboard found in IMG_6531.JPG: True
image pixel (2736, 1824)
Chessboard found in IMG_6532.JPG: True
image pixel (2736, 1824)
Chessboard found in IMG_6534.JPG: True
image pixel (2736, 1824)
Chessboard found in IMG_6535.JPG: True
image pixel (2736, 1824)
Chessboard found in IMG_6541.JPG: True
image pixel (2736, 1824)
Chessboard found in IMG_6542.JPG: True
image pixel (2736, 1824)
Chessboard foun

## 相机外参计算
在实验前拍摄一张完整的棋盘格pattern，检测格点的像素坐标，并与其在样品坐标系的坐标比对，算出相机外参。计算出的矩阵似乎是从相机位置移动到原点的变换，而非相机在坐标系中的位置，所以正负是相反的。
（目前误差有点大）

### 1 读取棋盘格角点

In [3]:
# import cv2
# import numpy as np

# # 读取图像
# image_path = 'D:\\labAAA\\printingLight\\process\\camera_set\\test250308\\camera_calib'
# img = cv2.imread(image_path)
# list_image_path = os.listdir(image_path)
# scale_factor = 2  # 你的缩放比例
# new_camera_matrix = camera_matrix.copy()
# new_camera_matrix[0, 0] *= scale_factor  # fx
# new_camera_matrix[1, 1] *= scale_factor  # fy
# new_camera_matrix[0, 2] *= scale_factor  # cx
# new_camera_matrix[1, 2] *= scale_factor  # cy
# print(new_camera_matrix)


### 使用手眼标定方法
1,读取四张图像的棋盘格角点

In [4]:
import cv2
import numpy as np

# 读取图像
image_path = 'D:\\labAAA\\printingLight\\process\\camera_set\\test250308\\camera_calib'
list_image_path = os.listdir(image_path)
scale_factor = 2  # 你的缩放比例
new_camera_matrix = camera_matrix.copy()
new_camera_matrix[0, 0] *= scale_factor  # fx
new_camera_matrix[1, 1] *= scale_factor  # fy
new_camera_matrix[0, 2] *= scale_factor  # cx
new_camera_matrix[1, 2] *= scale_factor  # cy
print(new_camera_matrix)

board_corners = []

for file in list_image_path:
    file_name = os.path.join(image_path, file)
    img = cv2.imread(file_name)
    w, h = 4, 4
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    ret, corners = cv2.findChessboardCorners(gray, (w, h), None)
    if ret:
        # 通过角点亚像素精确化，提高角点位置精度
        criteria = (cv2.TERM_CRITERIA_EPS + cv2.TERM_CRITERIA_MAX_ITER, 100,
                    0.0001)
        corners2 = cv2.cornerSubPix(gray, corners, (11, 11), (-1, -1),
                                    criteria)
        board_corners.append(corners2)
        # 可视化角点
        img_with_corners = cv2.drawChessboardCorners(img, (w, h), corners2,
                                                     ret)
        cv2.imwrite(
            f'D:\\labAAA\\printingLight\\process\\camera_set\\test250308\\corners\\{file}.png',
            img_with_corners)
    else:
        print("Chessboard corners not found.")

[[1.00164231e+04 0.00000000e+00 2.77132344e+03]
 [0.00000000e+00 1.00158486e+04 1.69584526e+03]
 [0.00000000e+00 0.00000000e+00 1.00000000e+00]]


2， 计算角点在样品坐标系下的坐标，并用solvepnp算外参

In [5]:
from scipy.spatial.transform import Rotation as R
import numpy as np



#从XYZABC输出变换矩阵
def kuka_to_transformation(pose):
    """将 KUKA 的点位信息转换为 4x4 变换矩阵"""
    # 平移向量
    t = np.array([pose[0],pose[1],pose[2]])

    # 计算旋转矩阵（ZYX顺序）
    rotation = R.from_euler('ZYX', [pose[3],pose[4],pose[5]], degrees=True)  # KUKA的ABC通常以度为单位
    R_matrix = rotation.as_matrix()
    # print(R_matrix)

    # 构造4x4变换矩阵
    T = np.eye(4)
    T[:3, :3] = R_matrix
    T[:3, 3] = t
    # print('T', T)
    return T

#从变换矩阵求旋转之后的棋盘格点在样品坐标系里的坐标
def transform_points(origin_points, T_1, T_2):

    # 将3D点转换为齐次坐标
    origin_points_h = np.hstack(
        [origin_points, np.ones((origin_points.shape[0], 1))])

    # 使用矩阵乘法进行坐标变换
    #去掉齐次坐标中的最后一列，转换回普通的3D坐标
    object_points_base_inv = np.dot(T_2, origin_points_h.T)
    object_points_o1 = np.dot(np.linalg.inv(T_1), object_points_base_inv).T
    return object_points_o1[:, :3]


#从XYZABC输出法兰在base里的变换矩阵
def tool2end(pose):
    T_t2f = kuka_to_transformation([-22.45, 6.745, 641.177, 0, 0, 0])
    T_t2b = kuka_to_transformation(pose)
    T_base2end = T_t2f @ np.linalg.inv(T_t2b)
    r = T_base2end[:3, :3]
    t = T_base2end[:3, 3]
    return r, t


In [14]:
#solvepnp
ends_r = []
ends_t = []
rvec_list = []
tvec_list = []
pose_list = [[1181.42, -2733.69, 1500, 57.72, -7.11, 78.67],
             [1181.42, -2733.69, 1500, 57.85, 5.18, 70.95],
             [1181.42, -2733.69, 1500, 57.45, 6.33, 81.56],
             [1181.42, -2733.69, 1500, 63.45, -7.25, 82.23]]
translation_vec = np.array([0.304,-0.944,-50.498])
grid_size = 90
grid_extend = [3,1,-1,-3]
grid_extend_j = [-3, -1, 1, 3]
#格点竖着排
points_col = np.array([(i * grid_size, j * grid_size, 0) for i in grid_extend_j
                   for j in grid_extend])
#格点横着排
points_row = np.array([(i * grid_size, j * grid_size, 0) for j in grid_extend
                       for i in grid_extend])
# print(points_col,'分割',points_row)
for i in range(len(board_corners)):
    if i == 1 or i == 2:  #竖排

        objectPoints = np.array(points_col + translation_vec, dtype=np.float32).reshape(-1, 1, 3)
    else:
        objectPoints = np.array(points_row + translation_vec, dtype=np.float32).reshape(-1, 1, 3)
    imagePoints = np.array(board_corners[i],
                           dtype=np.float32).reshape(-1, 1, 2)

    _, rvec, tvec, inliners = cv2.solvePnPRansac(objectPoints,
                                                 imagePoints.squeeze(),
                                                 new_camera_matrix,
                                                 distortion_coeffs)

    if not _:
        print("PnP 求解失败")

    rvec_list.append(rvec)
    tvec_list.append(tvec)
    rotation_matrix, _ = cv2.Rodrigues(rvec)

    print("Rotation Matrix:\n", rotation_matrix)
    print("Translation Vector:\n", tvec)
    img_points2, _ = cv2.projectPoints(objectPoints, rvec, tvec,
                                       new_camera_matrix, distortion_coeffs)

    # 如果 image_points 是 n x 1 x 2 的格式，调整为 n x 2 的格式
    image_points = np.array(imagePoints, dtype=np.float32).reshape(-1, 2)

    # 如果 imgpoints2 是 n x 1 x 2 的格式，调整为 n x 2 的格式
    img_points2 = np.array(img_points2, dtype=np.float32).reshape(-1, 2)
    # 计算重投影误差
    error = cv2.norm(image_points.squeeze(), img_points2.squeeze(),
                     cv2.NORM_L2) / len(img_points2)
    print("重投影误差 (Mean Reprojection Error):", error)

Rotation Matrix:
 [[-0.99341509  0.11450772  0.00379949]
 [-0.11340234 -0.9780193  -0.17498047]
 [-0.01632064 -0.17425911  0.98456457]]
Translation Vector:
 [[  33.18217718]
 [  31.17023508]
 [3193.19983758]]
重投影误差 (Mean Reprojection Error): 0.0921256772598158
Rotation Matrix:
 [[-0.99489916 -0.08914413 -0.04721204]
 [ 0.09937199 -0.94658826 -0.30675049]
 [-0.01734536 -0.30987736  0.9506183 ]]
Translation Vector:
 [[  32.46660381]
 [  29.12568429]
 [3190.26686441]]
重投影误差 (Mean Reprojection Error): 0.09505658370067464
Rotation Matrix:
 [[-0.99261562 -0.1178537  -0.02871815]
 [ 0.12056334 -0.9846132  -0.12649639]
 [-0.0133682  -0.12902465  0.99155128]]
Translation Vector:
 [[  33.00225207]
 [  32.29654037]
 [3192.35659495]]
重投影误差 (Mean Reprojection Error): 0.10363134887334777
Rotation Matrix:
 [[-0.98657624  0.12849485 -0.10077899]
 [-0.11703805 -0.986749   -0.11237667]
 [-0.11388339 -0.09907317  0.9885419 ]]
Translation Vector:
 [[  31.2554865 ]
 [  33.16710697]
 [3192.68806828]]
重投影误差 

In [15]:
#计算base在end中的位置

for pose in pose_list:
    r, t = tool2end(pose)
    
    ends_r.append(r)
    ends_t.append(t)



In [16]:
#标定

R_camera_corrected = [cv2.Rodrigues(r)[0] for r in rvec_list]
T_camera_corrected = [t.flatten() for t in tvec_list]
print('R_gripper2base\n', ends_r, '\nt_gripper2base\n', ends_t,
      '\nR_target2cam\n', R_camera_corrected, '\nt_target2cam\n', T_camera_corrected)

R_C2B, T_C2B = cv2.calibrateHandEye(
    R_gripper2base=ends_r, t_gripper2base=ends_t,
    R_target2cam=R_camera_corrected, t_target2cam=T_camera_corrected,method=cv2.CALIB_HAND_EYE_HORAUD
     # 其他方法：PARK, DANIILIDIS
)

# ==============================
# 4. 输出相机相对于机器人 Base 的位姿
# ==============================
print("🔹 相机相对机器人 Base 坐标系的位置 (T_C^B): ")
print("旋转矩阵 R_C^B:\n", R_C2B)
print("平移向量 T_C^B:\n", T_C2B)

R_gripper2base
 [array([[ 0.52995055,  0.8389471 ,  0.12377467],
       [-0.23091095,  0.0023149 ,  0.97297213],
       [ 0.81598563, -0.54420805,  0.19494886]]), array([[ 0.52996437,  0.84320009, -0.09028495],
       [-0.23093049,  0.24594021,  0.94137374],
       [ 0.81597113, -0.47804499,  0.32506015]]), array([[ 0.53475519,  0.83778325, -0.11025473],
       [-0.06504025,  0.17089909,  0.9831395 ],
       [ 0.84250024, -0.51856796,  0.14587881]]), array([[ 0.443405  ,  0.88739271,  0.12619897],
       [-0.17682992, -0.05142406,  0.98289712],
       [ 0.8787054 , -0.45813725,  0.13411589]])] 
t_gripper2base
 [array([ 1459.21512544, -1173.58214975, -2102.96412394]), array([ 1791.91458078,  -460.16543719, -2117.24465815]), array([ 1801.40131238,  -923.93925597, -1990.59188015]), array([ 1690.26058492, -1399.26771035, -1850.522208  ])] 
R_target2cam
 [array([[-0.99341509,  0.11450772,  0.00379949],
       [-0.11340234, -0.9780193 , -0.17498047],
       [-0.01632064, -0.17425911,  0.9845

In [17]:
T_cam = np.eye(4)
translation = T_C2B.flatten()
print(translation)
T_cam[:3,:3] = R_C2B
T_cam[:3,3] = T_C2B.flatten()

T_t2b = kuka_to_transformation([-170, -1800, 1500, 58, 0, 90])
tool2cam = (np.linalg.inv(T_cam)) @ T_t2b
print(tool2cam)



[-1437.01446232  -945.52841499  1514.42106172]
[[-9.99580194e-01 -1.97014437e-03 -2.89059694e-02  9.07589717e+00]
 [ 1.73583507e-03 -9.99965454e-01  8.12876898e-03  2.67431754e+01]
 [-2.89209857e-02  8.07518048e-03  9.99549082e-01  1.52802413e+03]
 [ 0.00000000e+00  0.00000000e+00  0.00000000e+00  1.00000000e+00]]


In [18]:
# 旋转矩阵和平移向量
R_t2c = tool2cam[:3, :3]  # 提取 Camera to Base 的旋转部分
t_t2c = tool2cam[:3, 3]   # 提取 Camera to Base 的平移部分


# 计算 rvec 和 tvec
rvec3, _ = cv2.Rodrigues(R_t2c)  # 旋转矩阵转换为旋转向量
tvec3 = t_t2c.reshape(3, 1)      # 确保 tvec 形状正确

In [19]:
corner = np.array([
    [90, 90, 0],
    [90, -90, 0],
    [-90, -90, 0],
    [-90, 90, 0]
], dtype=np.float32) + translation_vec#由于样品坐标系原点（也就是TCP）不在表面的中心，所以要计算格点在样品坐标系内的坐标
# corner = np.array([ 
#     [90, 90, 0],
#     [90, -90, 0],
#     [-90, -90, 0],
#     [-90, 90, 0]
# ], dtype=np.float32) 
#计算旋转以后的样品角点，这里只取了第一个做实验
T = kuka_to_transformation([-222.92, -1884.805, 1400, 32.616, 12.209, 70.802])#第一个路径点
#取标定外参时拍的另一张来实验
# T_test = kuka_to_transformation(1181.42, -2733.69, 1500, 57.72, -7.12, 78.67)#第一个标定点
# T = kuka_to_transformation(1181.42, -2733.69, 1500, 57.85, 5.18, 70.95)#第二个标定点
# T_test = kuka_to_transformation(1181.42, -2733.69, 1500, 63.45, -7.25, 82.23)
# corner_points = transform_points(corner,T_o1,T_test) + np.array([21.1771,-17.55,-6.745])
corner_points = transform_points(corner,T_t2b,T)

print('角点坐标',corner_points)
#计算投影在成像平面上的像素点
project_corner_points2, _ = cv2.projectPoints(corner_points, rvec3, tvec3, new_camera_matrix, distortion_coeffs)
print(project_corner_points2)

#绘制四边形
def draw_quadrilateral(image, points, output_path):
    """
    在图像上绘制四边形并保存结果

    参数:
    image: 输入图像
    points: 四个顶点坐标的列表 [(x1,y1), (x2,y2), (x3,y3), (x4,y4)]
    output_path: 输出图像的保存路径
    """
    # 将输入点转换为numpy数组
    pts = np.array(points, dtype=np.int32)

    # 在图像上绘制四边形
    cv2.polylines(image, [pts], isClosed=True, color=(0, 255, 0), thickness=5)

    # 保存结果
    cv2.imwrite(output_path, image)

# 使用示例:
if __name__ == "__main__":
    # 读取图像
    # image = cv2.imread('D:\\labAAA\\printingLight\\process\\camera_set\\test250308\\camera_calib\\IMG_6564.JPG')
    image = cv2.imread('D:\\labAAA\\printingLight\\process\\camera_set\\test250308\\images\\IMG_6567.JPG')
    


    points = project_corner_points2.tolist()
  
    # points = [corner_list[5], corner_list[6],corner_list[9],corner_list[10]]
    # 输出图像的保存路径
    # output_path = 'D:\\labAAA\\printingLight\\process\\camera_set\\test250308\\camera_calib\\draw_IMG_6564.JPG'
    output_path = 'D:\\labAAA\\printingLight\\process\\camera_set\\test250308\\images\\draw_IMG_6567.JPG'
    
    # 在图像上绘制四边形并保存
    draw_quadrilateral(image, project_corner_points2, output_path)

角点坐标 [[  25.67211633  -53.12600018  -25.52633816]
 [ -32.1800996  -219.27110032   12.53843312]
 [-191.1239361  -181.20500164  -62.87926354]
 [-133.27172017  -15.05990151 -100.94403482]]
[[[2666.39034876 2227.21781723]]

 [[3040.74694379 3295.72527161]]

 [[4153.94461219 3108.21381415]]

 [[3789.39689893 1980.78415538]]]
